In [3]:
import linguamind.linalg as la
import linguamind.nn as nn
import linguamind.nlp as nlp

from collections import Counter
import random

In [2]:
f = open('/Users/amberedmundson/Laboratory/datasets/harryPotter/harryPotterAsciiCleaned.txt','r')
raw = f.readlines()
f.close()

In [3]:
word2count = Counter()

sentences = list()
for paragraph in raw:
    sentence = list()
    for word in paragraph.split(" "):
        word2count[word] += 1
        sentence.append(word)
        if(word == '.'):
            if(len(sentence) > 4):
                sentences.append(sentence)
            sentence = list()
    if(len(sentence) > 4):
        sentences.append(sentence)
    sentence = list()

In [4]:
n_sample = 5
window = 5
# training_examples_context = list()
# training_examples_preds = list()

# for sentence in sentences_as_indices[0:10000]:
#     for pred_i in range(len(sentence)):
#         context = list()
#         predict = list() #only 0 is true index
#         for cont_i in range(window*2 +1):
#             if(0 == cont_i):
#                 continue
#             if(pred_i + cont_i - window >= 0):
#                 if(pred_i + cont_i - window < len(sentence)):
#                     context.append(sentence[pred_i + cont_i - window])

#         predict.append(sentence[pred_i])
#         for r in range(n_sample):
#             predict.append(random.randint(0,len(vocab)))
#         training_examples_context.append(context)
#         training_examples_preds.append(predict)

In [26]:
word2index = {}
vocab = list()

for word,cnt in word2count.most_common():
    vocab.append(word)
    word2index[word] = len(vocab)-1

sentences_as_indices = list()
for sentence in sentences:
    indicized_sentence = list()
    for word in sentence:
        indicized_sentence.append(word2index[word])
    sentences_as_indices.append(indicized_sentence)

vocab_size = len(word2count)+1
dim = 32

seed = la.Seed(1)

syn0 = nn.SparseLinearInput(vocab_size,dim)
syn0.weights.uniform(seed)
syn0.weights -= 0.5
syn0.weights /= 50

syn1 = nn.SparseLinearOutput(dim,vocab_size)
syn1.weights.zero()

layers = list()
layers.append(syn0)
layers.append(syn1)
# layers.append(nn.Relu(vocab_size))

mlp = nn.Sequential(layers)
criterion = nn.MSECriterion()
optim = nn.StochasticGradient(mlp,criterion)

In [16]:
cbow = nn.CBOW(sentences_as_indices,vocab_size,5,5)
cbow.target_values.resize(6)
cbow.next()

In [17]:
word2vec = {}
for word,index in word2index.items():
    word2vec[word] = syn0.weights[index]

In [18]:
optim.train(cbow,0.00001,1)

nan

In [19]:
sim = Counter()
for word in vocab:
    sim[word] = word2vec['her'].dot(word2vec[word])
sim.most_common()

[('her', 0.001315131550654769),
 ('eradication', 0.001315131550654769),
 ('confirming', 0.001315131550654769),
 ('insinuations', 0.001315131550654769),
 ('arrests', 0.001315131550654769),
 ('billowed', 0.0006805634475313127),
 ('weighted', 0.0006805634475313127),
 ('slug', 0.0006805634475313127),
 ("'yes", 0.0006805634475313127),
 ('frowning;', 0.0006805634475313127),
 ('dundee', 0.0006482075550593436),
 ('scuffling', 0.0006482075550593436),
 ('covertly', 0.0006482075550593436),
 ('blue;', 0.0006482075550593436),
 ('whirling', 0.0006091419490985572),
 ('monkeyish', 0.0006091419490985572),
 ('sickened', 0.0006091419490985572),
 ('injuries', 0.0006091419490985572),
 ('roast', 0.0006091419490985572),
 ('abject', 0.0006091419490985572),
 ('success', 0.0006083193002268672),
 ('glass-paneled', 0.0006083193002268672),
 ('lied!', 0.0006083193002268672),
 ('allies', 0.0006083193002268672),
 ('tremble', 0.0006083193002268672),
 ('dumbledores', 0.0006046612397767603),
 ('cup', 0.00060466123977676

# Forward Prop Modification for Windowing Layers

after you have confirmed that you are doing everything that word2vec does (exactly the same!!!)... such that you duplicate results, you should try this highly experimental thing

For CBOW, you should be able to tweak the output embedding based on how the inputs change and the previous gradient instead of regenerating it every time. If your window size is 5, this should give you close to a 5 * 2 = 10x speedup for the input window generation (which is where most of the parameters are going anyway)

# Windowing Object Passed to SGD

It should basically have a "get next training example" method.

# Negative Sampling Layers

### Full

In [36]:
%%timeit
optim.train(sentences_as_indices[0:new_limit],target,alpha=0.01,iterations=1)

1 loop, best of 3: 342 ms per loop


In [37]:
%%timeit
optim.train(sentences_as_indices,target,alpha=0.01,iterations=1)

1 loop, best of 3: 4.63 s per loop


# Optim 1: From training examples to windows

### None... just the loop

Note... previously 100

In [16]:
new_limit = int((100000/float(176216)) * 10000)

In [17]:
new_limit

5674

### None... just the loop

In [17]:
%%timeit
optim.train(sentences_as_indices[0:new_limit],target,alpha=0.01,iterations=1)

10 loops, best of 3: 28.4 ms per loop


In [18]:
%%timeit
optim.train(sentences_as_indices,target,alpha=0.01,iterations=1)

1 loop, best of 3: 395 ms per loop


### Forward Prop Only

In [17]:
%%timeit
optim.train(sentences_as_indices[0:new_limit],target,alpha=0.01,iterations=1)

1 loop, best of 3: 289 ms per loop


In [18]:
%%timeit
optim.train(sentences_as_indices,target,alpha=0.01,iterations=1)

1 loop, best of 3: 3.96 s per loop


### Forward and Back Prop

In [17]:
%%timeit
optim.train(sentences_as_indices[0:new_limit],target,alpha=0.01,iterations=1)

1 loop, best of 3: 353 ms per loop


In [18]:
%%timeit
optim.train(sentences_as_indices,target,alpha=0.01,iterations=1)

1 loop, best of 3: 4.66 s per loop


### Full

In [18]:
%%timeit
optim.train(sentences_as_indices[0:new_limit],target,alpha=0.01,iterations=1)

1 loop, best of 3: 387 ms per loop


In [19]:
%%timeit
optim.train(sentences_as_indices,target,alpha=0.01,iterations=1)

1 loop, best of 3: 5.2 s per loop


# Starting Point

### None... just the loop

In [14]:
%%timeit
optim.train(training_examples_context[0:100000],training_examples_preds[0:100000],target,alpha=0.01,iterations=1)

1 loop, best of 3: 424 ms per loop


### Forward Prop Only

In [13]:
%%timeit
optim.train(training_examples_context[0:100000],training_examples_preds[0:100000],target,alpha=0.01,iterations=1)

1 loop, best of 3: 5.41 s per loop


# Forward and Back Prop

In [13]:
%%timeit
optim.train(training_examples_context[0:100000],training_examples_preds[0:100000],target,alpha=0.01,iterations=1)

1 loop, best of 3: 6.36 s per loop


### Total Again

In [13]:
%%timeit
optim.train(training_examples_context[0:100000],training_examples_preds[0:100000],target,alpha=0.01,iterations=1)

1 loop, best of 3: 6.1 s per loop


### Total

In [15]:
%%timeit
optim.train(training_examples_context[0:100000],training_examples_preds[0:100000],target,alpha=0.01,iterations=1)

1 loop, best of 3: 6.08 s per loop


In [21]:
%%timeit
for input_indices,output_indices in training_examples[0:20000]:
    error = (optim.train(input_indices,output_indices,target,alpha=0.01,iterations=1))

1 loop, best of 3: 1.96 s per loop
